In [7]:
from DataGenerator import TimeSeriesGenerator, suddenDrift, incrementalDrift
from conditions import scenarios
from concurrent.futures import ProcessPoolExecutor
from grouped_series import SeriesGrouper, ExperimentTracker, Experiment
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.pipeline import Pipeline
from TimeSeriesGradientBoosting import TimeSeriesGradientBoosting
from xgboost import XGBRegressor
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from functools import partial
from sklearn.model_selection import train_test_split

In [ ]:
onehot_cols = ColumnTransformer([('one_hot_encoder', OneHotEncoder(
), make_column_selector(dtype_include=object))], remainder='passthrough')
xgb_pipe = Pipeline([
    ('onehot', onehot_cols),
    ('xgb', XGBRegressor())
])

In [2]:
def full_run():
    products = []
    customers = []
    for i in range(3):
        product = TimeSeriesGenerator(size=1460, amountOfVariables=7)
        customer = TimeSeriesGenerator(size=1460, amountOfVariables=3)
        products.append(product)
        customers.append(customer)
    thesis = ExperimentTracker(products, customers, scenarios())
    thesis.runExperiment(algorithm=xgb_pipe)
    return thesis


In [3]:
thesis = full_run()

In [16]:
results = thesis.resultsToDF()

BrokenProcessPool: A process in the process pool was terminated abruptly while the future was running or pending.

In [ ]:
results = thesis.resultsToDF()

In [11]:
results.groupby(["Drift type", "Drift time", "Drift magnitude"]).mean()

MSE      MAPE  \
Drift type        Drift time     Drift magnitude                           
Incremental Drift Fully observed Large               35.302898  0.011238   
                                 Small               11.994343  0.010942   
                  Half observed  Large              247.718579  0.012396   
                                 Small               23.932577  0.011664   
                  Unobserved     Large              248.124044  0.012376   
                                 Small               23.551588  0.011637   
No Drift          No Drift       No Drift            12.474393  0.011234   
Sudden Drift      Fully observed Large               25.707648  0.007849   
                                 Small               12.844181  0.009428   
                  Half observed  Large               73.448319  0.010060   
                                 Small               26.421749  0.012354   
                  Unobserved     Large            82864.245446  0.421967   
                                 Small             3932.818961  0.156704   

                                                      SMAPE  
Drift type        Drift time     Drift magnitude             
Incremental Drift Fully observed Large             1.106000  
                                 Small             1.089333  
                  Half observed  Large             1.276000  
                                 Small             1.167333  
                  Unobserved     Large             1.273333  
                                 Small             1.163333  
No Drift          No Drift       No Drift          1.116000  
Sudden Drift      Fully observed Large             0.783333  
                                 Small             0.940000  
                  Half observed  Large             1.006667  
                                 Small             1.229333  
                  Unobserved     Large            57.168000  
                                 Small            17.564667

In [8]:
results.sort_values("SMAPE", ascending=False)

,Dropped variable,Drift type,Drift magnitude,Variable importance,Drift time,MSE,MAPE,SMAPE
159,"[0, 3, 6]",Sudden Drift,Large,Important,Unobserved,176718.557925,0.609821,88.32
48,0,Sudden Drift,Large,Important,Unobserved,176323.756647,0.609134,88.18
11,None,Sudden Drift,Large,Important,Unobserved,175090.389513,0.607011,87.73
122,6,Sudden Drift,Large,Important,Unobserved,174741.004818,0.606425,87.61
85,3,Sudden Drift,Large,Important,Unobserved,174553.406806,0.605991,87.52
...,...,...,...,...,...,...,...,...
84,3,Sudden Drift,Large,Important,Half observed,12.154946,0.004205,0.42
120,6,Sudden Drift,Large,Important,Fully observed,12.157558,0.004205,0.42
121,6,Sudden Drift,Large,Important,Half observed,12.096642,0.004208,0.42
10,None,Sudden Drift,Large,Important,Half observed,12.149079,0.004189,0.42


In [9]:
results.to_csv("first_resultsWorking.csv")